# Model Evaluation Workflow :: NWM streamflow Analysis

<img src='../images/Eval_Analysis.svg' width=600>

## Essential Benchmark Components
This benchmark notebook will present a workflow which follows a canonical model for Essential Benchmark Components: 
1) A set of predictions and matching observation (the data); 
2) The domain (e.g. space or time) over which to benchmark;
3) A set of statistical metrics with which to benchmark. 

Let's get started.

## Step 0: Load libraries and configure Python computing environment

In [ ]:
import os
import logging
import pandas as pd

## Step 1: Load Data
<img src='../images/Eval_Analysis_Data.svg' width=360>

Essential Benchmark Components: 
1. **a set of predictions and matching observations;**
1. a domain over which to benchmark;
1. a set of statistical metrics with which to benchmark.

Finding and loading data is made easier for this particular workflow (the _streamflow_ variable), in that most of it has been 
pre-processed and stored in a cloud-friendly format.  That process is described in the [data preparation](./00-data-prep.ipynb)
notebook. We will proceed here using the already-prepared data for _streamflow_, which is included in the HyTEST [`intake`](https://intake.readthedocs.io/en/latest/index.html) catalog. 

In [ ]:
import intake
cat = intake.open_catalog(r'https://raw.githubusercontent.com/hytest-org/hytest/main/dataset_catalog/hytest_intake_catalog.yml')
print("Available datasets: \n", "\n".join(cat.keys()))

The above list represents the processed datasets available for benchmarking.  If a dataset
you want is not in that list, 
you can load the data manually via direct connection to 'on-prem' or S3 object storage. 
If you load data from a source other than this list, you can jump to [Step 2: Restrict to a Domain](#step-2-restrict-to-a-domain)

Note that the interesting datasets in the cataloged dataset above are duplicated: Some are `-onprem` 
and some are `-cloud`. Same data, but the storage location and access protocol will be different. You 
will definitely want to use the correct copy of the data for your computing environment.  
* `onprem` : Direct access via the `caldera` filesystem from _denali_ or _tallgrass_
* `cloud` : Network access via S3 bucket, suitable for consumption on cloud-hosted jupyter servers. You could also access using any network-attached computer, but the amount of data will likely saturate your connection.  Use in the cloud (e.g. ESIP QHub)

So... are you on-prem? 

In [ ]:
import platform
onprem = (platform.node() in ['denali', 'tallgrass'])  
# NOTE: these hostnames are not quite correct... this will always return not onprem.
if onprem:
    print("Yes : -onprem")
    obs_data_src='nwis-streamflow-usgs-gages-onprem'
    mod_data_src='nwm21-streamflow-usgs-gages-onprem'
else:
    print("Not onprem; use '-cloud' data source")
    obs_data_src='nwis-streamflow-usgs-gages-cloud'
    mod_data_src='nwm21-streamflow-usgs-gages-cloud'
print("Observed : ", obs_data_src)
print("Modeled  : ", mod_data_src)

In [ ]:
variable_of_interest = 'streamflow'
try:
    obs = cat[obs_data_src].to_dask()
    mod = cat[mod_data_src].to_dask()
except KeyError:
    print("Something wrong with dataset names.")
    raise

try:
    obs_data = obs[variable_of_interest]
    mod_data = mod[variable_of_interest].astype('float32')
except KeyError:
    print(f"{variable_of_interest} was not found in these data.")

obs_data.name = 'observed'
mod_data.name = 'predicted'    

## Step 2: Restrict to a Domain

<img src='../images/Eval_Analysis_Domain.svg' width=360>

Essential Benchmark Components: 
1. a set of predictions and matching observations;
1. **a domain over which to benchmark;**
1. a set of statistical metrics with which to benchmark.

Each benchmark domain is defined over specific bounds (typically space and/or time). 
Benchmark domain definitions can be defined within the notebook, or sourced from
elsewhere. For this example, we use the _Cobalt_ gages, avaliable for download on ScienceBase 
([Foks et al., 2022](https://doi.org/10.5066/P972P42Z)), as our 'spatial domain'.  

This is essentially a list of 
stream gages in which we are interested, along with some  metadata about that gage (watershed, 
reach code, etc).  We will use this as a spatial selector to restrict the original data to only 
those gages found within this benchmarking domain.

In [ ]:
cobalt = pd.read_csv(
    'https://www.sciencebase.gov/catalog/file/get/6181ac65d34e9f2789e44897?f=__disk__22%2F1a%2Fd2%2F221ad2fe9d95de17731ad35d0fc6b417a4b53ee1',
    dtype={'site_no':str, 'huc_cd':str, 'reachcode':str, 'comid':str, 'gagesII_class':str, 'aggecoregion': str}, 
    index_col='site_no'
    )
# Re-format the gage_id/site_no string value.  ex:   "1000000"  ==> "USGS-1000000"
cobalt.rename(index=lambda x: f'USGS-{x}', inplace=True)
print(f"{len(cobalt.index)} gages in this benchmark")
cobalt.head(2)

So now we have a domain dataset representing the stream gages (unique `site_no` values) identifying the locations making up the spatial domain of this benchmark. While we have good meta-data for these gages (lat/lon location, HUC8 code, etc), we really will only use the list of `site_no` values to select locations out of the raw data.

Now we can define our temporal domain or extent for the evaluation. Let's specify a certain time period below.

In [ ]:
# enter in start and end dates
start_time = '1983-10-01'
end_time='2016-12-31'

# from our available data, select a start and end time to restrict the temporal domain/extent of the assessment.
obs_data = obs_data.sel(time=slice(start_time,end_time))
mod_data = mod_data.sel(time=slice(start_time,end_time))

## Step 3: Define Metrics

<img src='../images/Eval_Analysis_Metrics.svg' width=360>

Essential Benchmark Components: 
1. a set of predictions and matching observations;
1. a domain over which to benchmark;
1. **a set of statistical metrics with which to benchmark.**

The specific metrics for the NWM "Standard Suite" have been codified for use in Python Jupyter notebooks
[here](../supplement/standard-benchmark-definitions.ipynb).  You can see examples of how these might be used in 
[this notebook](../supplement/standard-benchmark-usage.ipynb). 

You can use these pre-coded metrics or write your own. To use the Standard Suite, we can run their
definition notebook in the following cell. This will make its contents available to us in this notebook.

In [ ]:
%run ../supplement/standard-benchmark-definitions.ipynb


Whether you use these functions or your own, we need to put all metric computation into a single all-encompasing 
benchmarking function--a single call which can be assigned to each gage in our domain list. This sort of arrangement 
is well-suited to parallelism with `dask`. 

If this is done well, the process will benefit enormously from task parallelism -- each gage can be given its own 
CPU on which to run. After all are done, the various results will be collected and assembled into a composite dataset. 

To achieve this, we need a single 'atomic' function that can execute independently. It will take the gage identifier 
as input and return a list of metrics from the Standard Suite.

In [ ]:
## Wrapper function -- this func will be called once per gage_id, each call on its own dask worker
def compute_benchmark(gage_id):
    try:
        ## obs_data and mod_data should be globals...
        obs = obs_data.sel(gage_id=gage_id).load(scheduler='single-threaded').to_series()
        mod = mod_data.sel(gage_id=gage_id).load(scheduler='single-threaded').to_series().resample('1D', offset='5h').mean() 
        
        # make sure the indices match
        # this is a rough idea of 
        obs.index = obs.index.to_period('D')
        mod.index = mod.index.to_period('D')

        # merge obs and predictions; drop NaNs.
        gage_df = pd.merge(obs, mod, left_index=True, right_index=True).dropna(how='any')
        
        scores = pd.Series(
            data={
                'NSE': NSE(gage_df.observed, gage_df.predicted),
                'KGE': KGE(gage_df.observed, gage_df.predicted),
                'logNSE': logNSE(gage_df.observed, gage_df.predicted),
                'pbias': pbias(gage_df.observed, gage_df.predicted),
                'rSD': rSD(gage_df.observed, gage_df.predicted),
                'pearson': pearson_r(gage_df.observed, gage_df.predicted),
                'spearman': spearman_r(gage_df.observed, gage_df.predicted), 
                'pBiasFMS': pBiasFMS(gage_df.observed, gage_df.predicted),
                'pBiasFLV': pBiasFLV(gage_df.observed, gage_df.predicted),
                'pBiasFHV': pBiasFHV(gage_df.observed, gage_df.predicted)
            },
            name=gage_id,
            dtype='float64'
        )
        return scores
    
    except Exception as e:#<-- this is an extremely broad way to catch exceptions.  We only do it this way to ensure 
                          #    that a failure on one benchmark (for a single stream gage) will not halt the entire run. 
        logging.info("Benchmark failed for %s", gage_id)
        return None

Let's test to be sure this `compute_benchmark()` function will return data for a single gage

In [ ]:
compute_benchmark('USGS-01030350')

## Execute the Analysis 

We now need to set up a way to farm out this function, called once per gage ID, to workers in a 
parallel cluster. `dask` will do this using a dask '_bag_'.  

Read more about task parallelism with Dask and how we are using dask bags [here](https://docs.dask.org/en/stable/bag.html)


The cluster configuration is dependent on your AWS profile and credentials.  First, establish AWS identity: 

In [ ]:
### ### ### CONFIG CUSTOM CLUSTER HERE ### ### ###
aws_profile = 'usgs-scratch'
aws_region = 'us-west-2'
aws_endpoint = 'https://renc.osn.xsede.org'
  
import configparser
awsconfig = configparser.ConfigParser()
awsconfig.read(
    os.path.expanduser(r'~/.aws/credentials') # default location... if yours is elsewhere, change this.
)
# Set environment vars based on parsed awsconfig
os.environ['AWS_ACCESS_KEY_ID']     = awsconfig[aws_profile]['aws_access_key_id']    
os.environ['AWS_SECRET_ACCESS_KEY'] = awsconfig[aws_profile]['aws_secret_access_key']    
os.environ['AWS_S3_ENDPOINT']       = aws_endpoint
os.environ['AWS_DEFAULT_REGION']    = aws_region

try: 
    del os.environ['AWS_PROFILE']
except KeyError:
    pass

With identity established, we can spin up a cluster of workers, accessed via a Dask `gateway`

This next cell may take a moment.

In [ ]:
from dask_gateway import Gateway
gateway = Gateway()
options = gateway.cluster_options()
options.conda_environment='users/users-pangeo'
options.profile = 'Medium Worker'
options.environment_vars = dict(
    DASK_DISTRIBUTED__SCHEDULER__WORKER_SATURATION="1.0"
)
options.environment_vars.update(dict(os.environ))
cluster = gateway.new_cluster(options)
cluster.adapt(minimum=1, maximum=15)

# get the client for the cluster
client = cluster.get_client()
client.dashboard_link

In [ ]:
# If needed, you can adapt the max number of workers to speed up the problem.
# cluster.adapt(minimum=1, maximum=50)

In [ ]:
## Set up a dask bag with the contents being a list of the cobalt gages
import dask.bag as db

## Create a dask Bag from Python sequence.
bag = db.from_sequence(cobalt.index.tolist(), partition_size=30).map(compute_benchmark)

results = bag.compute() #<< Dispatch the workers

With that big task done, we don't need `dask` parallelism any more. Let's shut down the cluster:

In [ ]:
client.close(); del client
cluster.close(); del cluster

## Assemble the results
The `bag` now contains a collection of return values (one per call to `compute_benchmark()`).
We can massage that into a table/dataframe for easier processing: 

In [ ]:
r = [i for i in results if i is not None] # Drop entries where compute_benchmark failed
results_df = pd.concat(r, axis=1).T
results_df.index.name = 'site_no'
results_df

This dataframe/table can be saved to disk as a CSV. It will be used for visualizations in [other notebooks](NWM_Benchmark_Viz.ipynb).

In [ ]:
results_df.to_csv('NWM_v2.1_streamflow_example.csv') ##<--- change this to a personalized filename

This csv file will be used in the 06-benchmark-visualization notebooks.